# Постановка задачи

На сегодняшний день машинное обучение решает абсолютно разные задачи практически во всех областях нашей жизни. Платформа DataDriven организовывает контесты по машинному обучению для решения реальных задач общественных и некоммерческих организаций по всему миру, применяя результаты победителей. Данный проект призван решить проблемы водоснабжения в стране Танзания в Восточной Африке. Вода в Танзании добывается с помощью водокачек, и они часто выходят из строя, оставляя поселения без воды. Задача участников - по параметрам и статусу некоторого множества водокачек определить возможность поломки и необходимость обслуживания других водокачек. 

В данной задаче мы не только сделаем предсказания о состоянии водокачек, но и оценим, как качество данных влияет на предсказание.

Ссылка на контест: https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/



# Описание метода

Задача будет решаться с помощью градиентного бустинга - будем использовать одноимённый метод GradientBoosting из пакета sklearn.

## Практическая суть задачи

Пусть у нас существует некоторый набор пар {(𝑥𝑖,𝑦𝑖)}, i=1..n, где x - это признак, y - целевая переменная. Задача сводится к тому, чтобы восстановить функцию y=F(x) - функцию, минимизирующую функцию потерь L(x, F(x))

## Градиентный бустинг

Метод градиентного бустинга позволяет построить итоговую модель F(x) как взвешенную функцию слабых моделей Fm(x), m = 1..M. К каждой предыдущей сумме Fm-1(x) добавляется частная производная от функции L(x, Fm(x)) по Fm(x) со знаком минус - для минимизации потерь. Назовём эту производную hm(x). Необходимо также, чтобы функция L(x, Fm(x)) была дифференцируема в достаточно большой области. 

In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import GradientBoostingClassifier

Для работы будем использовать библиотеку Pandas. Выведем следующие данные:
TrainingX - множество признаков - набор данных о характеристиках водокачек
TrainingY - множество целевых переменных - набор данных о состоянии водокачек
TestX - множество признаков - набор данных о характеристиках водокачек, по которым требуется совершить предсказание.

In [183]:
TrainingX = pd.read_csv('./contest/trainingsetvalues.csv', sep=',')
TrainingY = pd.read_csv('./contest/trainingsetlabels.csv', sep=',')
TestX = pd.read_csv('./contest/testsetvalues.csv', sep=',')

Посмотрим, что представляют из себя данные о характеристиках водокачек:

In [184]:
print(type(TrainingX))

<class 'pandas.core.frame.DataFrame'>


In [185]:
TrainingX.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


# Пояснения к данным

Перечень столбцов с расшифровкой:
amount_tsh - Общий статический напор (количество воды, доступное для водокачки)
date_recorded - Дата ввода данных
funder - Кто финансировал колодец
gps_height - Глубина скважины
installer - Организация, вырывшая скважину
longitude - GPS-координата
latitude - GPS-координата
wpt_name - Название водоема, если он есть
num_private -
basin - Географическое расположение водоёма

subvillage - Географическое положение населённого пункта
region - Географическое положение региона
region_code - Код региона
district_code - Код района
lga - Географическое положение
ward - Географическое положение
population - Население вокруг скважины
public_meeting - Верно/Ложно
recorded_by - Групповой ввод этой строки данных
scheme_management - Кто управляет водокачкой
scheme_name - Кто управляет водокачкой
permit - Разрешен ли водозабор
construction_year - Год постройки водозабора
extraction_type - Тип добычи воды, которую использует водонапорная точка
extraction_type_group - Подвид добычи воды, которую использует водонапорная точка
extraction_type_class - Класс добычи воды, которую использует водонапорная точка
management - Как управляется водокачка
management_group - Как управляется водокачка
payment - Сколько стоит вода
payment_type - Сколько стоит вода
water_quality - Качество воды
quality_group - Качество воды
quantity - Количество воды
quantity_group - Количество воды
source - Источник воды
source_type - Источник воды
source_class - Источник воды
waterpoint_type - Вид водоема
waterpoint_type_group - Вид водоема


# Распределение данных по параметрам строковых типов
Чтобы включить в модель такие признаки, необходимо, чтобы было достаточно данных по каждому из них, и чтобы данные были достаточно равномерно распределены по ним. Для начала посмотрим, какие типы столбцов и в каком количестве у нас имеются:

In [5]:
TrainingX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

Как мы видим,  строковых значений в таблице 30, целочисленных - 7, вещественных - 3. Посмотрим на таблицу, содержащую только строковые типы данных:

In [6]:
TrainingX.select_dtypes(include=['object'])

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,2011-03-14,Roman,Roman,none,Lake Nyasa,Mnyusi B,Iringa,Ludewa,Mundindi,True,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,2013-03-06,Grumeti,GRUMETI,Zahanati,Lake Victoria,Nyamara,Mara,Serengeti,Natta,NaN,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,2013-02-25,Lottery Club,World vision,Kwa Mahundi,Pangani,Majengo,Manyara,Simanjiro,Ngorika,True,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,2013-01-28,Unicef,UNICEF,Zahanati Ya Nanyumbu,Ruvuma / Southern Coast,Mahakamani,Mtwara,Nanyumbu,Nanyumbu,True,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,2011-07-13,Action In A,Artisan,Shuleni,Lake Victoria,Kyanyamisa,Kagera,Karagwe,Nyakasimbi,True,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,2013-05-03,Germany Republi,CES,Area Three Namba 27,Pangani,Kiduruni,Kilimanjaro,Hai,Masama Magharibi,True,...,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
59396,2011-05-07,Cefa-njombe,Cefa,Kwa Yahona Kuvala,Rufiji,Igumbilo,Iringa,Njombe,Ikondo,True,...,annually,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
59397,2011-04-11,NaN,NaN,Mashine,Rufiji,Madungulu,Mbeya,Mbarali,Chimala,True,...,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
59398,2011-03-08,Malec,Musa,Mshoro,Rufiji,Mwinyi,Dodoma,Chamwino,Mvumi Makulu,True,...,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump


## Практическая оценка качества данных
Для начала попробуем оценить качество данных, построив модель напрямую. Оценим точность предсказания, разделив данные на две части: на одной обучим модель, по второй оценим точность.

Делим данные на два датафрейма:

In [113]:
TrainX1 = TrainingX[:29700]
TrainY1 = TrainingY[:29700]
TestX2 = TrainingX[29700:]
TestY2 = TrainingY[29700:]
TestY = TrainingY[29700:]

Создаём объект модели:

In [114]:
model = GradientBoostingClassifier(
        n_estimators=500,
        max_depth=5,
        #subsample=0.5,
        max_features="auto"
        # random_state=seed
    )

In [115]:
TestY2

,id,status_group
29700,62962,1
29701,35938,1
29702,5107,0
29703,64811,1
29704,8958,1
...,...,...
59395,60739,0
59396,27263,0
59397,37057,0
59398,31282,0


Включаем в выборку все столбцы, приводимые к float напрямую, и приводим типы данных:

In [116]:
TrainX1.dtypes == object


id                       False
amount_tsh               False
date_recorded             True
funder                    True
gps_height               False
installer                 True
longitude                False
latitude                 False
wpt_name                  True
num_private              False
basin                     True
subvillage                True
region                    True
region_code              False
district_code            False
lga                       True
ward                      True
population               False
public_meeting            True
recorded_by               True
scheme_management         True
scheme_name               True
permit                    True
construction_year        False
extraction_type           True
extraction_type_group     True
extraction_type_class     True
management                True
management_group          True
payment                   True
payment_type              True
water_quality             True
quality_

Все столбцы, типом которых НЕ является object, мы можем привести к типу float. 

In [117]:
ctrainX = TrainX1[['id',
 'amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'num_private',
 'region_code',
 'district_code',
 'population',
 'construction_year'
                  ]]
#ctrainX = ctrainX.values()
ctrainX = ctrainX.to_numpy()
ctrainX = ctrainX.astype(float)
ctrainX

array([[6.9572e+04, 6.0000e+03, 1.3900e+03, ..., 5.0000e+00, 1.0900e+02,
        1.9990e+03],
       [8.7760e+03, 0.0000e+00, 1.3990e+03, ..., 2.0000e+00, 2.8000e+02,
        2.0100e+03],
       [3.4310e+04, 2.5000e+01, 6.8600e+02, ..., 4.0000e+00, 2.5000e+02,
        2.0090e+03],
       ...,
       [6.0922e+04, 0.0000e+00, 0.0000e+00, ..., 4.0000e+00, 0.0000e+00,
        0.0000e+00],
       [5.6395e+04, 0.0000e+00, 0.0000e+00, ..., 1.0000e+00, 0.0000e+00,
        0.0000e+00],
       [7.2779e+04, 0.0000e+00, 1.5060e+03, ..., 7.0000e+00, 1.4000e+02,
        2.0010e+03]])

In [118]:
TrainY1.replace(to_replace="functional", value=0, inplace=True)
TrainY1.replace(to_replace="non functional", value=1, inplace=True)
TrainY1.replace(to_replace="functional needs repair", value=2, inplace=True)

ctrainY = TrainY1
ctrainY = ctrainY.to_numpy()
ctrainY = ctrainY[:, 1]
ctrainY = ctrainY.astype(float)
ctrainY

c:\python3\lib\site-packages\pandas\core\frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


array([0., 0., 0., ..., 1., 1., 0.])

In [119]:
ctestX = TestX2[['id',
 'amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'num_private',
 'region_code',
 'district_code',
 'population',
 'construction_year'
                  ]]
ctestX = ctestX.to_numpy()
ctestX = ctestX.astype(float)


Строим модель:

In [120]:
model.fit(ctrainX,ctrainY)

#predictions = model.predict_proba(testX)[:,1]
predictions = model.predict_proba(ctestX)
predictions

array([[0.16324187, 0.79217712, 0.04458101],
       [0.49407471, 0.41001585, 0.09590944],
       [0.93835256, 0.04746945, 0.01417799],
       ...,
       [0.5522829 , 0.19316773, 0.25454937],
       [0.45815469, 0.4832003 , 0.05864501],
       [0.40366304, 0.56320941, 0.03312755]])

Мы получили таблицу вероятностей каждого из трех состояний. Выберем максимально вероятные состояния:

In [121]:
answ = predictions[:, 0].astype(int)
for ii in range (0, len(predictions)):
    answ[ii] = predictions[ii].argmax();
answ

array([1, 0, 0, ..., 0, 1, 1])

Теперь запишем их в датафрейм и сравним с действительными данными:

In [122]:
TestY['status_group'] = answ
TestY

c:\python3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,status_group
29700,62962,1
29701,35938,0
29702,5107,0
29703,64811,0
29704,8958,1
...,...,...
59395,60739,0
59396,27263,0
59397,37057,0
59398,31282,1


In [123]:
TestY2

,id,status_group
29700,62962,1
29701,35938,1
29702,5107,0
29703,64811,1
29704,8958,1
...,...,...
59395,60739,0
59396,27263,0
59397,37057,0
59398,31282,0


In [124]:
TestY2.replace(to_replace="functional", value=0, inplace=True)
TestY2.replace(to_replace="non functional", value=1, inplace=True)
TestY2.replace(to_replace="functional needs repair", value=2, inplace=True)
TestY2

c:\python3\lib\site-packages\pandas\core\frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,id,status_group
29700,62962,1
29701,35938,1
29702,5107,0
29703,64811,1
29704,8958,1
...,...,...
59395,60739,0
59396,27263,0
59397,37057,0
59398,31282,0


In [125]:
(TestY2['status_group'] == TestY['status_group']).sum()

20287

In [131]:
acc = 100 * (TestY2['status_group'] == TestY['status_group']).sum() / len(TestY2)
print(acc,'%')

68.3063973063973 %


Точность предсказания составила 68.3%! Это неплохо, учитывая, что данные ещё никак не оптимизировались. Займёмся их оптимизацией.

# Оптимизация данных
Для начала избавимся от дубликатов в фрейме:

In [186]:
TrainingX = TrainingX.drop_duplicates(subset = ['amount_tsh', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group'],keep = 'first')
TrainingX

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,...,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
59396,27263,4700.0,2011-05-07,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,...,annually,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
59397,37057,0.0,2011-04-11,NaN,0,NaN,34.017087,-8.750434,Mashine,0,...,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
59398,31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,...,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump


## Статический напор

In [244]:
amount = TrainingX.loc[TrainingX['amount_tsh'] != 0]
amount

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
2,34310,25.0,2013,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
5,9944,20.0,2011,Mkinga Distric Coun,0,DWE,39.172796,-4.765587,Tajiri,0,...,per bucket,salty,salty,enough,enough,other,other,unknown,communal standpipe multiple,communal standpipe
11,50409,200.0,2013,Danida,1062,DANIDA,35.770258,-10.574175,Kwa Alid Nchimbi,0,...,on failure,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
16,48451,500.0,2011,Unicef,1703,DWE,34.642439,-9.106185,Kwa John Mtenzi,0,...,monthly,soft,good,dry,dry,river,river/lake,surface,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59385,34473,500.0,2012,Jaica,1327,JAICA CO,33.951681,-2.021854,Kwa Marunda,0,...,monthly,salty,salty,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
59387,26640,100.0,2013,0,25,0,39.176480,-6.957098,Kwa Maliba,0,...,per bucket,soft,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe
59394,11164,500.0,2011,World Bank,351,ML appro,37.634053,-6.124830,Chimeredya,0,...,monthly,soft,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe
59395,60739,10.0,2013,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,...,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


Для большинства водокачек этот параметр не указан. Включать в модель не будем.

## Дата ввода данных
Оценим разброс дат ввода данных. 

Для начала нам надо привести даты к такому виду, чтобы их можно было использовать для внесения в модель.

In [192]:
TrainingX['date_recorded'] = TrainingX['date_recorded'].map(lambda x: str(x)[:4])
TrainingX['date_recorded'] = TrainingX['date_recorded'].astype(object).astype(int)
TrainingX

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,2013,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,...,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
59396,27263,4700.0,2011,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,...,annually,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
59397,37057,0.0,2011,NaN,0,NaN,34.017087,-8.750434,Mashine,0,...,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
59398,31282,0.0,2011,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,...,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump


Оценим диапазон дат записей:

In [194]:
val = TrainingX['date_recorded'].value_counts()
val

2011    28638
2013    24260
2012     6411
2004       30
2002        1
Name: date_recorded, dtype: int64

Большинство записей сделано в период с 2011 по 2013 годы, но есть весьма старые записи, выделяющиеся из выборки. Мы можем либо исключить из этого множества 31 водокачку, данные о которых записаны в 2002 и 2004 годы, либо же использовать их, если о них достаточно много данных о годах постройки. Посмотрим на разброс данных по годам постройки:

In [195]:
val = TrainingX['construction_year'].value_counts()
val

0       20671
2010     2644
2008     2612
2009     2533
2000     2091
2007     1587
2006     1471
2003     1285
2011     1256
2004     1123
2012     1084
2002     1075
1978     1037
1995     1014
2005     1011
1999      979
1998      966
1990      954
1985      944
1996      811
1980      807
1984      779
1982      744
1994      738
1972      708
1974      671
1997      644
1992      640
1993      608
2001      540
1988      520
1983      488
1975      437
1986      433
1976      414
1970      411
1991      323
1989      316
1987      302
1981      238
1977      202
1979      192
1973      184
2013      176
1971      142
1960      101
1967       88
1963       85
1968       75
1969       59
1964       40
1962       30
1961       21
1965       19
1966       17
Name: construction_year, dtype: int64

Видим, что значительная часть годов постройки не указана. Оценим количество неуказанных годов:

In [196]:
print(int(len(TrainingX[TrainingX['construction_year'] == 0])/len(TrainingX)*100.0), "%")

34 %


Немало. Теперь посмотрим, насколько полные данные о годе постройки у нас есть для старых записей

In [204]:
val = TrainingX.loc[TrainingX['date_recorded'] < 2005]
val1 = val.loc[val['construction_year'] > 0]
val1

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
762,3995,0.0,2004,Dhv Moro,277,DWE,36.516088,-8.097199,Hospital,0,...,unknown,unknown,unknown,unknown,unknown,shallow well,shallow well,groundwater,hand pump,hand pump
1189,20122,0.0,2004,Cct,777,Regional Water,39.130087,-10.572385,Kwa Mzee Mkata,0,...,never pay,soft,good,enough,enough,spring,spring,groundwater,other,other
3446,53174,0.0,2004,Hesawa,1362,HESAWA,34.516282,-1.732201,Musati Primary School Dp No.7,0,...,unknown,soft,good,enough,enough,spring,spring,groundwater,communal standpipe multiple,communal standpipe
8729,20198,0.0,2004,Government Of Tanzania,86,DWE,38.959776,-5.247278,Kwa Bi Shoga,0,...,never pay,salty abandoned,salty,dry,dry,shallow well,shallow well,groundwater,hand pump,hand pump
10441,55069,20.0,2004,Plan Int,307,DDCA,38.768656,-7.298419,Mianzi Primary School,0,...,per bucket,soft,good,enough,enough,spring,spring,groundwater,hand pump,hand pump
13366,48759,100.0,2004,Mgaya,1331,Mgaya,34.290885,-1.699609,Maro Mgaya,0,...,per bucket,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
22754,51853,0.0,2004,Hesawa,1167,DWE,32.881651,-1.935430,Nyamarasi,0,...,never pay,salty,salty,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
23373,20534,50.0,2004,Tasaf And Lga,239,TASAFcitizen and LGA,39.272736,-11.019000,Juhudi,0,...,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe multiple,communal standpipe
27501,53086,500.0,2004,Kkkt,1611,Villagers,34.900561,-8.873813,Kwa Barnabasi Kilumile,0,...,on failure,soft,good,enough,enough,hand dtw,borehole,groundwater,hand pump,hand pump
32619,9195,0.0,2004,Caltus,1856,Cultus,31.539761,-7.983106,Kwa Pambajila,0,...,never pay,soft,good,dry,dry,spring,spring,groundwater,communal standpipe,communal standpipe


более чем у половины старых записей есть данные о годе постройки. Не будем их исключать.

## Кто финансировал колодец

Посчитаем количество всех учередителей:

In [242]:
val = TrainingX['funder'].value_counts()
val.to_csv('./contest/res.csv',index = True)

In [243]:
val

Government Of Tanzania    9068
Danida                    3114
Hesawa                    2198
Rwssp                     1372
World Bank                1349
                          ... 
Rarymond Ekura               1
Justine Marwa                1
Municipal Council            1
Afdp                         1
Samlo                        1
Name: funder, Length: 1897, dtype: int64

1897 наименований и большой разброс по их количеству... Сочтём это плохим признаком и не будем включать в модель. 

## Глубина скважины

Глубина скважины может быть как выше уровня моря (положительная), так и ниже (отрицательная). Определить полноту данных можно по количеству скважин, глубина которых равняется нулю

In [246]:
val = TrainingX.loc[TrainingX['gps_height'] == 0]
val

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
4,19728,0.0,2011,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
5,9944,20.0,2011,Mkinga Distric Coun,0,DWE,39.172796,-4.765587,Tajiri,0,...,per bucket,salty,salty,enough,enough,other,other,unknown,communal standpipe multiple,communal standpipe
6,19816,0.0,2012,Dwsp,0,DWSP,33.362410,-3.766365,Kwa Ngomho,0,...,never pay,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
7,54551,0.0,2012,Rwssp,0,DWE,32.620617,-4.226198,Tushirikiane,0,...,unknown,milky,milky,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
8,53934,0.0,2012,Wateraid,0,Water Aid,32.711100,-5.146712,Kwa Ramadhan Musa,0,...,never pay,salty,salty,seasonal,seasonal,machine dbh,borehole,groundwater,hand pump,hand pump
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59384,72148,0.0,2011,Concern,0,CONCERN,30.667805,-2.483710,Chamkube,0,...,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
59392,40607,0.0,2011,Government Of Tanzania,0,Government,33.009440,-8.520888,Benard Charles,0,...,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
59393,48348,0.0,2012,Private,0,Private,33.866852,-4.287410,Kwa Peter,0,...,per bucket,soft,good,insufficient,insufficient,dam,dam,surface,other,other
59397,37057,0.0,2011,NaN,0,NaN,34.017087,-8.750434,Mashine,0,...,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump


Отсутствует больше трети данных, и субъективно глубина относительно уровня моря - далеко не самый существенный признак. Исключим его.

## Установщик

In [249]:
val = TrainingX['installer'].value_counts()
val.to_csv('./contest/res.csv',index = True)

Видим, что распределение установщиков неравномерное, данные о некоторых отсутствуют, многие наименования фактически дублируются под разными названиями. Этот признак плохо подходит для модели, включать его не будем.

## Координаты

Субъективно координаты являются достаточно важным признаком. Для начала оценим, сколько у нас нулевых координат:

In [250]:
val = TrainingX.loc[TrainingX['longitude'] == 0]
val

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
21,6091,0.0,2013,Dwsp,0,DWE,0.0,-2.000000e-08,Muungano,0,...,unknown,unknown,unknown,unknown,unknown,shallow well,shallow well,groundwater,hand pump,hand pump
53,32376,0.0,2011,Government Of Tanzania,0,Government,0.0,-2.000000e-08,Polisi,0,...,unknown,unknown,unknown,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
168,72678,0.0,2013,Wvt,0,WVT,0.0,-2.000000e-08,Wvt Tanzania,0,...,other,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
177,56725,0.0,2013,Netherlands,0,DWE,0.0,-2.000000e-08,Kikundi Cha Wakina Mama,0,...,unknown,soft,good,enough,enough,shallow well,shallow well,groundwater,other,other
253,13042,0.0,2012,Hesawa,0,DWE,0.0,-2.000000e-08,Kwakisusi,0,...,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59189,62177,0.0,2011,Dwsp,0,DWE,0.0,-2.000000e-08,Wazazo,0,...,other,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
59208,3631,0.0,2013,Dwsp,0,DWE,0.0,-2.000000e-08,Mtakuja,0,...,unknown,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
59295,60843,0.0,2011,Rwssp,0,DWE,0.0,-2.000000e-08,Maendeleo,0,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
59324,748,0.0,2013,World Vision,0,World Vision,0.0,-2.000000e-08,Mwazwilo,0,...,unknown,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump


На фоне всего массива данных данных с незаполненными координатами немного. Включить в модель их всё же стоит, но необходимо заменить все нулевые координатами средними.

In [254]:
mean = TrainingX[TrainingX['longitude'] != 0]
mean.mean()
mean

c:\python3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,2013,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,...,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
59396,27263,4700.0,2011,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,...,annually,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
59397,37057,0.0,2011,NaN,0,NaN,34.017087,-8.750434,Mashine,0,...,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
59398,31282,0.0,2011,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,...,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
